Note: This is a new version of a notebook that I originally worked up elsewhere. It now operates using the pymodelcat package that I started. I've changed up the documentation accordingly.

Now that we have a [container](https://www.sciencebase.gov/catalog/item/5e8de96182cee42d134687cc) for model items in ScienceBase that we can operate against, it opens up all kinds of interesting possibilities for "robots" to do some work for us. This notebook explores what we might be able to get from the web links added into the mix. Theoretically, those represent a wealth of meaty material to help flesh out the model catalog as a useful resource. We can write some code that can go do some gathering, and then decide if there is anything that we can consistently bring back in to flesh out the items. Because we're doing this in code and this whole thing is experimental, we can be reasonably safe in writing information back into ScienceBase for use and evaluation. We just need to keep track of the parts of ScienceBase Items where we want to "cede control to the robots" and the parts we want to manage in some other way.

There are several different strategies for leveraging those links to get more information together. One interesting dynamic would be to simply index everything we find on subsequent landing pages and even spidering into the contents like any search engine. I've proposed in the past that ScienceBase could do this writ large, providing directed search engine functionality to go after linked content in meaningful ways.

For this exercise, I'm focusing on a couple ways of fishing for structured metadata. This little robot is essentially the machine we talk about when we say things about "machine-readable" metadata or other content. A couple of potential strategies occur to me:

* Content negotiation - Some web pages and applications accessible over HTTP enable content negotiation, which is a method for the requestor to negotiate the structure or substance of the content that is returned in a response. It can also do things like specify languages that content should be returned in. This is all based on what the server/application providing the content will actually support. You can't force the system to give you something it isn't prepared to deliver. Systems like ScienceBase and some other USGS platforms do support content negotiation with a couple different machine readable options, so it's worth trying to see what we might be able to use.
* Structured metadata - There are a whole variety of ways that web systems have worked out to embed structured, machine-readable metadata within the HTML content delivered through web pages and web apps. Many of these coalesce around the schema.org set of content specifications to give us some degree of consistency to work from. One cool thing about these techniques is that they can be implemented right within the primary vehicle for web content delivery - the web page viewed by humans. Perhaps the coolest thing from a data science perspective is that the specifications and encoding methods really encourage explicit semantics (meaning we don't have to guess about what words mean), use of persistent identifiers and linking to associated registries (so we don't have to guess about disambiguating things), and linking between concepts that relate (so we can go after a wealth of information and tie it all together into a network/graph).

Of these two, structured metadata probably offers us the most promise in terms of richness of content, consistency in resulting data, and overall ease of use. Unfortunately, the uptake of these methods in USGS is pretty abysmal, including in ScienceBase where we've not updated our use of schema.org metadata in 8 years or so.

As a last resort, we could fall back on painful web scraping methods where we essentially parse HTML content and try to extract useful information. These are painful, because every one would essentially be a mostly custom affair that is probably more trouble than its worth. I do show a small bit of that here just for demonstration purposes.

In [1]:
from pymodelcat.catbuilder import Catbuilder
from IPython.display import display
import random

Most of the functional logic to operate this robot is now housed in the Catbuilder class of the pymodelcat Python package. We instantiate the class as the cb object here and then operate the various functions. I'll explain what the functions do, but please reference the package code as needed.

In [2]:
cb = Catbuilder()

In this codeblock, I use a get_models function to retrieve the model items in the model catalog. Right now, this returns a scaled down data structure that has just the stuff we care about working with most. We'll need to work up some options in this function for future use if we want to continue having a bit of a model-specific abstraction on the ScienceBase Item model via this API.

We could just run through and check links in real time, but a) I don't want to retrieve the same URL more than once and b) ScienceBase gets cranky with too many requests. Since interacting with the links is what we're doing here and may well be a regular thing, I have this function give us back a simple list of every unique webLink URI in the collection. This takes care of any cases where more than one model item might be referencing the same link. Later, if I decide to use the information I've collected, I can come back to my model list and figure out which ScienceBase Items the information applies to.

At this point, I'm not concerned about what type of link we're dealing with. I can again come back and work on using the title of the web links (e.g., "Model Reference Link") to make decisions on what to do with what I find. In looking through the links, however, and the information that does come back on some of them, we might need to do a little more work to better classify just what these links mean when it comes to using content from their pages in a meaningful way (more on that later). To give a sense of what's returned here, I output a couple of examples.

In [3]:
models, unique_links = cb.get_models()

display(models[5])

display(unique_links[5:10])

{'id': '5e8f338782cee42d134adf0c',
 'title': 'PULSE',
 'webLinks': [{'type': 'webLink',
   'typeLabel': 'Web Link',
   'uri': 'https://water.usgs.gov/ogw/pulse/',
   'rel': 'related',
   'title': 'Model Reference Link',
   'hidden': False}]}

['https://www.usgs.gov/software/modpath-a-particle-tracking-model-modflow',
 'https://doi.org/10.5066/P9E6INWZ',
 'https://wwwbrr.cr.usgs.gov/projects/GWC_chemtherm/software.htm',
 'https://pubs.usgs.gov/sir/2011/5190/',
 'https://water.usgs.gov/cgi-bin/man_wrdapp?daflow(1)']

With our list of unique URLs, we can now go out and figure out if we have anything interesting to work with. Our workflow in this example does all of that in one go so that each URL only gets hit once to see what we can figure out. In this stage, I am essentially gathering some content and adding what I find to a data structure for each URL. I can then go back and process that in different ways to pull out what I want to use, but for now, I'm only taking it this far to examine for possibilities.

The main gather_model_meta() function from pymodelcat works through the following logical components into a very rough and notional function to process each given URL. Depending on how useful this information proves to be, we would want to refine this to something more real and manageable.

* Content negotiation is a whole topic in its own right that could use some careful consideration and more advanced methdos than I use here. As I said above, structured metadata holds a lot more interest and potential. I'm only trying to see if there is some kind of JSON response as the simplest thing to deal with. With the requests package, I can add in an Accept header that will still give me text/html content to work with if my Accept header is not accepted. For now, I just use a try block to see if I can shove the JSON content into my data structure. It's crude, but effective for our immediate purpose.
* For the structured metadata part, I've found the [extruct](https://github.com/scrapinghub/extruct) package from the ScrapingHub folks to be one of the most reliable, but it doesn't deal with quite all of the derivations folks have employed on embedding structured metadata in web pages. In this instance, I just try for anything we can get to evaluate for use.
* The final thing threw in here as something of a Hail Mary if a basic web page meta scraper. It parses HTML content using the Python BeautifulSoup package and returns the page title and any named meta tags with content. This can sometimes yield a reasonable description depending on a lot of factors. This is a really terrible way to try and go about things for any kind of consisency given the vagaries of content management systems, legacy content, and all kinds of factors. But it's some other stuff to look at. I put this into a function that could be built upon further if it turns out this is actually a reasonable source to think about. There are also some more robust alternatives to this like the ScrapingHub AutoExtract API that we could think about.

Running this kind of process in a big loop isn't a great method for some eventual production application. There are all kinds of considerations in doing this kind of thing in terms of having our robots be polite web crawlers and not freaking system administrators out with too many "weird" requests and how to optimize this kind of thing to check back in routinely over time for updated information. We could certainly paralellize this in a number of ways from launching Lambdas on the cloud to multithreading, and that would pull together data nice and quick. For now, working the relatively small number of links through in a loop is fine for demonstration and evaluation purposes.

Note: If you are fiddling with this and want to see what things look like before trying every URL, add something like ```[:5]``` to the end of the "unique_links" list in the for loop to only run a handful of URLs through the process.

In [4]:
%%time
link_eval_results = list()
for link in unique_links:
    link_eval_results.append(cb.gather_model_meta(link))

CPU times: user 36.7 s, sys: 1.12 s, total: 37.8 s
Wall time: 5min 33s


Now's the fun part: analyzing the data and seeing if there's anything we want to use. Well, it would be fun, except our results aren't actually all that consistent or robust.

We have very few cases where we were able to pull any useful structured metadata in at all, so our most promising route for a consistent and powerful method is stymied by the fact that most of the systems behind these links haven't implemented that method. As a little bit of a side tangent, someone really ought to be encouraging this type of thing across the USGS and perhaps leading by example. Beyond supporting what we're trying to do here, these methods would go a long way to improving how USGS content presents itself on the web, how we might influence search rankings, and how we might influence the various knowledge graph efforts toward recognizing USGS as an authority on some subjects.

Content negotiation doesn't give us a whole lot of results either with a couple of notable exceptions. Somewhat by design, anything that points at a DOI link should respond to some type of accept header that will give us DOI metadata as a response.

As clunky as it is, meta tag scraping may still be a viable option to try to work from (if we can avoid encouraging bad behavior). Meta tags are really not designed for modern robots as they do not really have capacity for explicit semantics to understand what the intent of the meta tags should be. It's all a matter of convention and usage within a particular context, and that has to be unraveled and dealt with in some fashion.

Beyond the information structure, some of the research questions I would want to pursue include the following:

* Can we legitimately use anything we bring back in these processes in a meaningful way to add more depth to our model catalog?
* Can we use titles from any of these sources provide more than model acronyms or short names in our ScienceBase Items?
* Do any of the descriptions make sense to serve as an abstract for the concept of the model as cataloged, or are they for some specific aspect of the modeling system?
* Does our initial notional way of type classifying web links (e.g., Model Reference Link) help us in determining what information we can use?
* How might the extended information from model related assets like model output data or software code be encoded into the model items to add value without adding confusion?

These tests give you a quick run down on what things we found and can go look into further. The list of dictionaries could be put into a number of other data structures for further analysis, but I find it easy enough just to work with a little bit of list comprehension at this point. I created a little helper function that incorporates the basic logic on how to tease out the interesting bits to look at.

So, to get a look at the data and start thinking through if and how we might use stuff we can pull back from these different sources and methods, here is a quick dump of the numbers of things we have in each category of potentially useful data and a random output of one record in that "bucket."

In [5]:
for bucket in cb.meta_bucket_list:
    this_bucket = cb.subset_model_meta(link_eval_results, bucket)
    print(bucket, len(this_bucket))
    display(this_bucket[random.randint(0,len(this_bucket)-1)])

all 158


{'url': 'https://doi.org/10.5066/P9Q8JGAO',
 'json_response': None,
 'structured_data': None,
 'meta_content': {'title': 'Model Data Sets for 1DTempPro and VFLUX simulation experiments to Determine Groundwater Seepage in the Indian River Lagoon, Florida'}}

json_response 23


{'url': 'https://www.sciencebase.gov/catalog/item/57c44124e4b0f2f0cebc8a2a',
 'json_response': {'link': {'rel': 'self',
   'url': 'https://www.sciencebase.gov/catalog/item/57c44124e4b0f2f0cebc8a2a'},
  'relatedItems': {'link': {'url': 'https://www.sciencebase.gov/catalog/itemLinks?itemId=57c44124e4b0f2f0cebc8a2a',
    'rel': 'related'}},
  'id': '57c44124e4b0f2f0cebc8a2a',
  'identifiers': [{'type': 'DOI',
    'scheme': 'https://www.sciencebase.gov/vocab/category/item/identifier',
    'key': 'doi:10.5066/F7KW5D4H'}],
  'title': 'Water-quality and streamflow datasets used in the Weighted Regressions on Time, Discharge, and Season (WRTDS) models to determine trends in the Nation’s rivers and streams, 1972-2012',
  'summary': 'In 1991, the U.S. Geological Survey (USGS) began a study of more than 50 major river basins across the Nation as part of the National Water-Quality Assessment (NAWQA) project of the National Water-Quality Program. One of the major goals of the NAWQA project is to de

microdata 19


...

{'url': 'http://regclim.coas.oregonstate.edu/visualization/rcd/',
 'json_response': None,
 'structured_data': {'microdata': [],
  'json-ld': [],
  'opengraph': [],
  'microformat': [{'type': ['h-feed'],
    'properties': {'name': ['Regional and Global Climate']},
    'children': [{'type': ['h-entry'], 'properties': {}}]}],
  'rdfa': [{'@id': 'http://regclim.coas.oregonstate.edu/visualization/rcd/#masthead',
    'http://www.w3.org/1999/xhtml/vocab#role': [{'@id': 'http://www.w3.org/1999/xhtml/vocab#banner'}]},
   {'@id': 'http://regclim.coas.oregonstate.edu/visualization/rcd/#main',
    'http://www.w3.org/1999/xhtml/vocab#role': [{'@id': 'http://www.w3.org/1999/xhtml/vocab#main'}]},
   {'@id': 'http://regclim.coas.oregonstate.edu/visualization/rcd/',
    'https://api.w.org/': [{'@id': 'http://regclim.coas.oregonstate.edu/wp-json/'}]}]},
 'meta_content': {'title': 'Regional Climate Downloader – Regional and Global Climate',
  'viewport': 'width=device-width, initial-scale=1',
  'generato

rdfa 73


{'url': 'https://doi.org/10.5066/F7JS9NHH',
 'json_response': None,
 'structured_data': {'microdata': [],
  'json-ld': [],
  'opengraph': [],
  'microformat': [],
  'rdfa': [{'@id': 'https://www.mbr-pwrc.usgs.gov:443/bbs/bbs2014.html#navbar',
    'http://www.w3.org/1999/xhtml/vocab#role': [{'@id': 'http://www.w3.org/1999/xhtml/vocab#banner'}]}]},
 'meta_content': {'title': 'Untitled Document'}}

meta_content 158


{'url': 'https://www.usgs.gov/software/thornthwaite-monthly-water-balance-model',
 'json_response': None,
 'structured_data': {'microdata': [],
  'json-ld': [],
  'opengraph': [],
  'microformat': [],
  'rdfa': [{'@id': 'https://www.usgs.gov/software/thornthwaite-monthly-water-balance-model#navbar',
    'http://www.w3.org/1999/xhtml/vocab#role': [{'@id': 'http://www.w3.org/1999/xhtml/vocab#banner'}]}]},
 'meta_content': {'title': 'Thornthwaite Monthly Water Balance Model',
  'viewport': 'width=device-width, initial-scale=1.0',
  '': 'text/html; charset=utf-8',
  'description': 'Full model documentation, including both a description of the physical process modeling methodology and the graphical user interface can be accessed from the button below. A demonstration input data file (necessary to run the model) is available with the model download.Read more about the model in USGS Open-File Report 2007-1088:\xa0A Monthly Water-Balance Model Driven by a',
  'abstract': 'The Thornthwaite wate

To unwind all this and get back to the original model item that contained a URL that we evaluated, we would go look for that URL in our original models data structure that we created at the beginning. Once we get to this point and have some data to feed back, we will look at more elegant ways of dealing with this. The following code simply picks a random evaluated link item in a particular category and finds the first corresponding model item where that link came from. There are better ways of stitching this all together. 

In [6]:
example_link_item = cb.subset_model_meta(link_eval_results, "microdata")[random.randint(0,len(cb.subset_model_meta(link_eval_results, "microdata"))-1)]
display(example_link_item)
model_item = next((m for m in models if example_link_item["url"] in [l["uri"] for l in m["webLinks"]]), None)
display(model_item)

{'url': 'https://pubs.er.usgs.gov/publication/sir20195045',
 'json_response': None,
 'structured_data': {'microdata': [{'type': 'http://schema.org/ScholarlyArticle',
    'properties': {'name': 'The Hydrologic System of the South Florida Peninsula: Development and Application of the Biscayne and Southern Everglades Coastal Transport (BISECT) Model',
     'author': ['Eric D. Swain', 'Melinda A. Lohmann', 'Carl R. Goodwin'],
     'sameAs': 'https://doi.org/10.3133/sir20195045',
     'description': 'Abstract\n\nFirst posted September 26, 2019\n\nFor additional information, contact:\n\nDirector, Caribbean-Florida Water Science Center\nU.S. Geological Survey\n4446 Pet Lane, Suite 108\nLutz, FL 33559\n\nThe Biscayne and Southern Everglades Coastal Transport (BISECT) model was developed by the U.S. Geological Survey under the Greater Everglades Priority Ecosystem Studies Initiative to evaluate, both separately and in conjunction, the likely effects on surface-water stages and flows, hydroperio

{'id': '5e8de97682cee42d134687d5',
 'title': 'BISECT',
 'webLinks': [{'type': 'webLink',
   'typeLabel': 'Web Link',
   'uri': 'https://pubs.er.usgs.gov/publication/sir20195045',
   'rel': 'related',
   'title': 'Model Reference Link',
   'hidden': False}]}

### A note about formal, standardized metadata
I can imagine a question coming up about why we would not pursue tying in formal, structured metadata where it exists into this process, hitting things like FGDC CSDGM records and using the more robust metadata there in this process. It's a valid question and would be worth pursuing. The reasons I did not address that yet in this exploration include the following:

* The web links that are often found, provided, and included so far in our exploratory work usually go to some type of information landing page meant for human consumption. It often takes some additional sleuthing or web scraping or other means to discover a link to a formal metadata record. Interpretive conventions could be applied in our case here to make assumptions that a "catalog.data.gov" link is going to have a formal metadata record, and with a little rewriting in the URL or further API search, we might find it to exploit. However, I'd argue that the best thing to do is for those catalog systems to put all of that metadata content right into one form or the other of landing page embedded structured metadata that can be read by processes like this as well as by humans.
* For right now, we really don't need all that much metadata content to make our model items quite a bit better. Limiting to what we might get through this kind of process keeps things simple and focuses on the highest level metadata elements that we probably care most about at the moment.
* The harder issues to work through have to do with the substance of the content and whether or not it adds value in this context. Context in which metadata are produced matters immensely in many cases. Metadata produced in a FGDC CSDGM record for releasing a product is done for one purpose while metadata put into a advertisement page for that product online may be quite different. At this stage, the metadata advertising the product is probably the most useful context for our immediate purposes.